In [1]:
import cv2
import numpy as np
import tensorflow as tf
import pickle
import mediapipe as mp
import os
from datetime import datetime

# --- Load All Models and Encoders ---
try:
    # Load k-NN assets
    with open('knn_model.pkl', 'rb') as f:
        knn_model = pickle.load(f)
    with open('label_encoder_knn.pkl', 'rb') as f:
        label_encoder_knn = pickle.load(f)
    
    # Load CNN assets
    cnn_model = tf.keras.models.load_model('cnn_model.keras')
    with open('label_encoder_cnn.pkl', 'rb') as f:
        label_encoder_cnn = pickle.load(f)
        
    print("✅ All models loaded successfully!")
except FileNotFoundError:
    print("Error: Could not find model files. Please train both models first.")
    exit()

# --- Re-use Feature Extraction and Preprocessing Functions ---
def extract_landmarks(image):
    # (Same function from knn_model.ipynb)
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        landmarks = []
        wrist_coords = hand_landmarks.landmark[0]
        for lm in hand_landmarks.landmark:
            landmarks.extend([lm.x - wrist_coords.x, lm.y - wrist_coords.y, lm.z - wrist_coords.z])
        return np.array(landmarks)
    return None

def preprocess_for_cnn(image, image_size=(64, 64)):
    # (Same preprocessing from cnn_model.ipynb)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized_image = cv2.resize(gray_image, image_size)
    normalized_image = resized_image.astype("float32") / 255.0
    final_image = np.expand_dims(normalized_image, axis=[0, -1])
    return final_image

# --- Main Application ---
cap = cv2.VideoCapture(0)
capture_folder = "captures"
os.makedirs(capture_folder, exist_ok=True)

while True:
    ret, frame = cap.read()
    if not ret: break
    cv2.imshow('Webcam - Press SPACE to Capture, Q to Quit', frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key == ord(' '): # Spacebar
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        capture_path = os.path.join(capture_folder, f"capture_{timestamp}.jpg")
        cv2.imwrite(capture_path, frame)
        print(f"\nImage saved to '{capture_path}'")
        
        saved_image = cv2.imread(capture_path)
        if saved_image is not None:
            print("\n--- Running Predictions ---")
            
            # k-NN Prediction
            landmarks = extract_landmarks(saved_image)
            if landmarks is not None:
                knn_pred_index = knn_model.predict([landmarks])[0]
                knn_pred_label = label_encoder_knn.inverse_transform([knn_pred_index])[0]
                print(f"k-NN (MediaPipe) Prediction: {knn_pred_label}")
            else:
                print("k-NN: Could not detect hand landmarks.")

            # CNN Prediction
            cnn_image = preprocess_for_cnn(saved_image)
            cnn_pred_probs = cnn_model.predict(cnn_image)
            cnn_pred_index = np.argmax(cnn_pred_probs)
            cnn_pred_label = label_encoder_cnn.inverse_transform([cnn_pred_index])[0]
            print(f"CNN (Pixels) Prediction: {cnn_pred_label}")
            
            print("-------------------------")

cap.release()
cv2.destroyAllWindows()

✅ All models loaded successfully!


2025-07-07 18:16:40.826 python[13087:108505] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.



Image saved to 'captures/capture_20250707_181703.jpg'

--- Running Predictions ---


I0000 00:00:1751905023.842898  108505 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1751905023.862661  109230 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751905023.874677  109228 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751905023.933831  109223 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


k-NN (MediaPipe) Prediction: che
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
CNN (Pixels) Prediction: fe
-------------------------

Image saved to 'captures/capture_20250707_181808.jpg'

--- Running Predictions ---
k-NN (MediaPipe) Prediction: qe
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
CNN (Pixels) Prediction: re
-------------------------


...


Image saved to 'captures/capture_20250707_181814.jpg'

--- Running Predictions ---
k-NN (MediaPipe) Prediction: qe
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
CNN (Pixels) Prediction: re
-------------------------


I0000 00:00:1751905094.365798  108505 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3
W0000 00:00:1751905094.379224  110730 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751905094.393791  110728 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.



Image saved to 'captures/capture_20250707_181815.jpg'

--- Running Predictions ---
k-NN (MediaPipe) Prediction: qe
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
CNN (Pixels) Prediction: re
-------------------------


I0000 00:00:1751905095.631109  108505 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3
W0000 00:00:1751905095.647078  110781 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751905095.659760  110781 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


: 